In [1]:
%cd ..

/home/ec2-user/uclresearchanalysis


In [2]:
import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config
import math
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from IPython.display import display
import numpy as np
import pandas as pd
import time
import traceback

('Reading config file from location: '
 '/home/ec2-user/uclresearchanalysis/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/uclresearchanalysis/followers',
                    'friends': '/home/ec2-user/uclresearchanalysis/friends'},
          'cwd': '/home/ec2-user/uclresearchanalysis/data/givenchy',
          'ml': '/home/ec2-user/uclresearchanalysis/data/givenchy/pickle',
          'networkx': {'all': '/home/ec2-user/uclresearchanalysis/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/uclresearchanalysis/data/givenchy/pickle/networkx_friends.dat',


In [3]:
unique_users = config.load_users_dataframe()

Loading data file from path /home/ec2-user/uclresearchanalysis/data/givenchy/pickle/users.dat
'Loaded 5973 entires'


In [4]:
unique_users = unique_users.sample(frac=1)
unique_users.head()

,text,time_lapsed,order,created_at,description,favourites_count,followers_count,followers_json,friends_count,friends_json,...,source_index,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count
1459,RT @TCM_Party: Influential designer Hubert de ...,123.32,1460.0,2015-11-22 18:53:07,"Cats, history, music, The Walking Dead, vintag...",1226,176,"[964762249339400192, 238112587, 10038803657090...",693,"[4715537477, 455313759, 252569527, 122482029, ...",...,1432,81,2,95.19,841,4.382878,0.209275,1.457788,0.002378,0.824019
3774,"RIP to a fashion god, Hubert de Givenchy.\n\nI...",439.17,3775.0,2012-04-01 21:02:00,dopamine enthusiast | cmu alum | author of A R...,3040,242,"[552404184, 263793586, 221838349, 990986293344...",247,"[2372254448, 1614375631, 351335161, 2846309177...",...,None,3774,0,0,2171,3.455090,0.111469,1.400276,0.003224,0.113772
2469,RT @IAMFASHlON: One of the most influential fa...,212.97,2470.0,2014-09-30 17:57:06,instagram; @gracexkidd,12275,428,"[761483864732729344, 1017563923, 1005355975673...",865,"[2488433658, 756786980202119168, 7607287212298...",...,21,21,1,195.97,1259,1.551231,0.339952,9.749801,0.001589,0.687053
1648,RT @IAMFASHlON: One of the most influential fa...,138.65,1649.0,2011-07-15 22:22:09,I am not my tweets. I am not these 140 charact...,1915,386,"[1249311642, 1001022967621537792, 2871816759, ...",320,"[4133313808, 993092256725946368, 1408410739, 2...",...,48,21,2,121.65,2432,4.246711,0.158717,0.787418,0.000411,0.131579
1616,"RT @MEENAVOGUEE: The iconic, Hubert de Givench...",135.30,1617.0,2009-06-04 01:25:23,Electric Lady.,393,839,"[988461878518145025, 291394684, 350854813, 321...",261,"[199722567, 1352311544, 150995011, 73544651617...",...,972,21,4,118.3,3204,19.367353,0.261860,0.122659,0.001248,0.081461


In [5]:
intervals = [30]
# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [6]:
unique_users.loc[0, 'followers_count'] + unique_users.loc[0, 'friends_count']

147762

In [7]:
in_degree_centrality = list(unique_users.friends_count)
out_degree_centrality = list(unique_users.followers_count)
degree_centrality = in_degree_centrality + out_degree_centrality

In [8]:
infectedNetworkStats = {}
for interval in intervals:
    statsDictionary = {}
    for timestamp in range(0, 24*60+1, interval):
        infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
        statsDictionary[timestamp] = {}
        statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
        statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
        statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
    infectedNetworkStats[interval] = statsDictionary
infectedNetworkStats

{30: {0: {'totalNodeInfected': 1,
   'totalFollowers': 143119,
   'totalFriends': 4643},
  30: {'totalNodeInfected': 116,
   'totalFollowers': 3014932,
   'totalFriends': 154438},
  60: {'totalNodeInfected': 599,
   'totalFollowers': 3755464,
   'totalFriends': 538640},
  90: {'totalNodeInfected': 956,
   'totalFollowers': 5951130,
   'totalFriends': 807857},
  120: {'totalNodeInfected': 1414,
   'totalFollowers': 7601061,
   'totalFriends': 1115589},
  150: {'totalNodeInfected': 1796,
   'totalFollowers': 8338643,
   'totalFriends': 1391046},
  180: {'totalNodeInfected': 2144,
   'totalFollowers': 12436777,
   'totalFriends': 1669491},
  210: {'totalNodeInfected': 2438,
   'totalFollowers': 12885888,
   'totalFriends': 1857873},
  240: {'totalNodeInfected': 2727,
   'totalFollowers': 13216334,
   'totalFriends': 2093647},
  270: {'totalNodeInfected': 2939,
   'totalFollowers': 13911835,
   'totalFriends': 2214913},
  300: {'totalNodeInfected': 3142,
   'totalFollowers': 14108529,
   '

In [12]:
def process_data(interval, start_index, end_index):
    features = {
        'label': [],
        # Spr: Spreading
        'Spr_generation0': [],
        'Spr_generation-1': [],
        'Spr_timeSinceSeed0': [],
        'Spr_timeSinceSeed-1': [],
        'Spr_totalNodeInfected': [],
        'Spr_totalFollowers': [],
        'Spr_totalFriends': [],
        'Spr_followersCentrality0': [],
        'Spr_followersCentrality-1': [],
        'Spr_friendsCentrality0': [],
        'Spr_friendsCentrality-1': [],
        # UsM: User metadata
        'UsM_deltaDays': [], # Y
        'UsM_statusesCount': [], # Y
        'UsM_followersCount': [], # Y
        'UsM_favouritesCount': [], # Y
        'UsM_friendsCount': [], # Y
        'UsM_listedCount': [], # Y
        'UsM_normalizedUserStatusesCount': [], # Y
        'UsM_normalizedUserFollowersCount': [], # Y
        'UsM_normalizedUserFavouritesCount': [], # Y
        'UsM_normalizedUserListedCount': [], # Y
        'UsM_normalizedUserFriendsCount': [], # Y              
        'UsM_deltaDays0': [], # Y
        'UsM_statusesCount0': [], # Y
        'UsM_followersCount0': [], # Y
        'UsM_favouritesCount0': [], # Y
        'UsM_friendsCount0': [], # Y
        'UsM_listedCount0': [], # Y
        'UsM_normalizedUserStatusesCount0': [], # Y
        'UsM_normalizedUserFollowersCount0': [], # Y
        'UsM_normalizedUserFavouritesCount0': [], # Y
        'UsM_normalizedUserListedCount0': [], # Y
        'UsM_normalizedUserFriendsCount0': [], # Y
        'UsM_deltaDays-1': [], # Y
        'UsM_statusesCount-1': [], # Y
        'UsM_followersCount-1': [], # Y
        'UsM_favouritesCount-1': [], # Y
        'UsM_friendsCount-1': [], # Y
        'UsM_listedCount-1': [], # Y
        'UsM_normalizedUserStatusesCount-1': [], # Y
        'UsM_normalizedUserFollowersCount-1': [], # Y
        'UsM_normalizedUserFavouritesCount-1': [], # Y
        'UsM_normalizedUserListedCount-1': [], # Y
        'UsM_normalizedUserFriendsCount-1': [], # Y
        # TwM: Tweet metadata
        'TwM_t0': [],
        'TwM_t-1': [],
        'TwM_tCurrent': [],
        # Nw: Network
        'Nw_nNodes': [], # Y
        'Nw_degree': [], # Y
        'Nw_inDegree': [], # Y
        'Nw_outDegree': [], # Y
        'Nw_degree0': [],
        'Nw_inDegree0': [],
        'Nw_outDegree0': [],
        'Nw_degree-1': [],
        'Nw_inDegree-1': [],
        'Nw_outDegree-1': [],
        # Stat: Statistical
        'Stat_average_kOut': [],
        'Stat_average_t': [],
        'Stat_average_deltaDays': [],
        'Stat_average_statusesCount': [],
        'Stat_average_followersCount': [],
        'Stat_average_favouritesCount': [],
        'Stat_average_friendsCount': [],
        'Stat_average_listedCount': [],
        'Stat_average_normalizedUserStatusesCount': [],
        'Stat_average_normalizedUserFollowersCount': [],
        'Stat_average_normalizedUserFavouritesCount': [],
        'Stat_average_normalizedUserListedCount': [],
        'Stat_average_normalizedUserFriendsCount': [],                
        'Stat_max_kOut': [],
        'Stat_min_kOut': []
    }

    with tqdm(total=len(list(unique_users[start_index: end_index].iterrows()))) as pbar: 
        for index, user_row in unique_users[start_index: end_index].iterrows():
            try:
                if len(user_row['source_candidates']) > 0 and ((user_row['source_index'] is not None) or math.isnan(user_row['time_lapsed'])):          
                    source_candidates = user_row['source_candidates']
                    source_first = source_candidates[0]
                    source_first_row = unique_users.loc[source_first]
                    source_first_time_lapsed = source_first_row.time_lapsed
                    start_bar = int(source_first_time_lapsed / interval) + 1
                    end_bar = min((user_row['time_lapsed'] / interval) + 1, (24*60/interval))
                    bars = list(np.arange(start_bar * interval, end_bar * interval, interval))
                    number_of_bars = len(bars)

                    for current_time in bars:
                        # all sources up to the current time
                        sources = [x for x in source_candidates if unique_users.loc[x].time_lapsed <= current_time]
                        sources_dataframe = unique_users.loc[sources]
                        degreeCentralityList = list(degree_centrality[i] for i in sources)
                        inDegreeCentralityList = list(in_degree_centrality[i] for i in sources)
                        outDegreeCentralityList = list(out_degree_centrality[i] for i in sources)
                        degreeList = list(unique_users.loc[i, 'followers_count'] + unique_users.loc[i, 'friends_count']  for i in sources)
                        timeList = [current_time - unique_users.loc[x].time_lapsed for x in sources]

                        first_source_index = sources[0]
                        first_source_row = unique_users.loc[first_source_index]
                        last_source_index = sources[-1]
                        last_source_row = unique_users.loc[last_source_index]

                        #Extraction
                        if math.isnan(user_row['time_lapsed']):
                            features['label'].append(0)
                        else:
                            features['label'].append(int(current_time >= user_row['time_lapsed']))
                        # Spr: Spreading
                        features['Spr_generation0'].append(source_first_row['generation'])
                        features['Spr_generation-1'].append(last_source_row['generation'])
                        features['Spr_timeSinceSeed0'].append(source_first_row['time_since_seed'])
                        features['Spr_timeSinceSeed-1'].append(last_source_row['time_since_seed'])
                        features['Spr_totalNodeInfected'].append(infectedNetworkStats[interval][current_time]['totalNodeInfected'])
                        features['Spr_totalFollowers'].append(infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['Spr_totalFriends'].append(infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['Spr_friendsCentrality0'].append(infectedNetworkStats[interval][current_time]['totalFriends'])#in_degree[source_first]/
                        features['Spr_friendsCentrality-1'].append(infectedNetworkStats[interval][current_time]['totalFriends'])#in_degree[last_source_index]/
                        features['Spr_followersCentrality0'].append(infectedNetworkStats[interval][current_time]['totalFollowers'])#out_degree[source_first]/
                        features['Spr_followersCentrality-1'].append(infectedNetworkStats[interval][current_time]['totalFollowers'])#out_degree[last_source_index]/
                        # UsM: User metadata                    
                        features['UsM_deltaDays'].append(user_row['user_created_days'])
                        features['UsM_statusesCount'].append(user_row['statuses_count'])
                        features['UsM_followersCount'].append(user_row['followers_count'])
                        features['UsM_favouritesCount'].append(user_row['favourites_count'])
                        features['UsM_friendsCount'].append(user_row['friends_count'])
                        features['UsM_listedCount'].append(user_row['listed_count'])
                        features['UsM_normalizedUserStatusesCount'].append(user_row['normalized_statuses_count'])
                        features['UsM_normalizedUserFollowersCount'].append(user_row['normalized_followers_count'])
                        features['UsM_normalizedUserFavouritesCount'].append(user_row['normalized_favourites_count'])
                        features['UsM_normalizedUserListedCount'].append(user_row['normalized_listed_count'])
                        features['UsM_normalizedUserFriendsCount'].append(user_row['normalized_friends_count'])                 
                        features['UsM_deltaDays0'].append(source_first_row.user_created_days)
                        features['UsM_statusesCount0'].append(source_first_row.statuses_count)
                        features['UsM_followersCount0'].append(source_first_row.followers_count)
                        features['UsM_favouritesCount0'].append(source_first_row.favourites_count)
                        features['UsM_friendsCount0'].append(source_first_row.friends_count)
                        features['UsM_listedCount0'].append(source_first_row.listed_count)
                        features['UsM_normalizedUserStatusesCount0'].append(source_first_row.normalized_statuses_count)
                        features['UsM_normalizedUserFollowersCount0'].append(source_first_row.normalized_followers_count)
                        features['UsM_normalizedUserFavouritesCount0'].append(source_first_row.normalized_favourites_count)
                        features['UsM_normalizedUserListedCount0'].append(source_first_row.normalized_listed_count)
                        features['UsM_normalizedUserFriendsCount0'].append(source_first_row.normalized_friends_count)
                        features['UsM_deltaDays-1'].append(last_source_row.user_created_days)
                        features['UsM_statusesCount-1'].append(last_source_row.statuses_count)
                        features['UsM_followersCount-1'].append(last_source_row.followers_count)
                        features['UsM_favouritesCount-1'].append(last_source_row.favourites_count)
                        features['UsM_friendsCount-1'].append(last_source_row.friends_count)
                        features['UsM_listedCount-1'].append(last_source_row.listed_count)
                        features['UsM_normalizedUserStatusesCount-1'].append(last_source_row.normalized_statuses_count)
                        features['UsM_normalizedUserFollowersCount-1'].append(last_source_row.normalized_followers_count)
                        features['UsM_normalizedUserFavouritesCount-1'].append(last_source_row.normalized_favourites_count)
                        features['UsM_normalizedUserListedCount-1'].append(last_source_row.normalized_listed_count)
                        features['UsM_normalizedUserFriendsCount-1'].append(last_source_row.normalized_friends_count) 
                        # TwM: Tweet metadata
                        features['TwM_t0'].append(round(timeList[0], 1))
                        features['TwM_t-1'].append(round(timeList[-1], 1))
                        features['TwM_tCurrent'].append(current_time)
                        # Nw: Network
                        features['Nw_nNodes'].append(len(sources))
                        features['Nw_degree'].append(degree_centrality[index])
                        features['Nw_inDegree'].append(in_degree_centrality[index])
                        features['Nw_outDegree'].append(out_degree_centrality[index])
                        features['Nw_degree0'].append(degree_centrality[source_first])
                        features['Nw_inDegree0'].append(in_degree_centrality[source_first])
                        features['Nw_outDegree0'].append(out_degree_centrality[source_first])
                        features['Nw_degree-1'].append(degree_centrality[last_source_index])
                        features['Nw_inDegree-1'].append(in_degree_centrality[last_source_index])
                        features['Nw_outDegree-1'].append(out_degree_centrality[last_source_index])
                        # Stat: Statistical
                        features['Stat_average_kOut'].append(round(mean(degreeList), 1))
                        features['Stat_average_t'].append(round(mean(timeList), 1))
                        features['Stat_average_deltaDays'].append(sources_dataframe.user_created_days.mean())
                        features['Stat_average_statusesCount'].append(sources_dataframe.statuses_count.mean())
                        features['Stat_average_followersCount'].append(sources_dataframe.followers_count.mean())
                        features['Stat_average_favouritesCount'].append(sources_dataframe.favourites_count.mean())
                        features['Stat_average_friendsCount'].append(sources_dataframe.friends_count.mean())
                        features['Stat_average_listedCount'].append(sources_dataframe.listed_count.mean())
                        features['Stat_average_normalizedUserStatusesCount'].append(sources_dataframe.normalized_statuses_count.mean())
                        features['Stat_average_normalizedUserFollowersCount'].append(sources_dataframe.normalized_followers_count.mean())
                        features['Stat_average_normalizedUserFavouritesCount'].append(sources_dataframe.normalized_favourites_count.mean())
                        features['Stat_average_normalizedUserListedCount'].append(sources_dataframe.normalized_listed_count.mean())
                        features['Stat_average_normalizedUserFriendsCount'].append(sources_dataframe.normalized_friends_count.mean())
                        features['Stat_max_kOut'].append(max(degreeList))
                        features['Stat_min_kOut'].append(min(degreeList))
                    processed_dataframe = pd.DataFrame(features)
            except Exception as ex:
                print(ex)
                traceback.print_exc()
                print(source_first_row, interval)
            pbar.update(1)
    return processed_dataframe

In [13]:
number_of_processes = multiprocessing.cpu_count()
print('Will start {} processes'.format(number_of_processes))
for interval in intervals:
    with Pool(number_of_processes) as pool:
        parameters = []
        number_of_users = len(unique_users.index)
        task_size = math.ceil(number_of_users/number_of_processes)
        for i in range(number_of_processes):
            start_index = i * task_size
            end_index = min((i + 1) * task_size, number_of_users)
            parameters.append((interval, start_index, end_index))
        dataframe_results = pool.starmap(process_data, parameters)
        
    result = pd.DataFrame()
    result = result.append(dataframe_results)
    config.dump_ml_data(result, interval)
    print('extracted {} of rows'.format(len(result.index)))
    display(result)

Will start 16 processes


100%|██████████| 374/374 [01:10<00:00,  5.83it/s]


Dumping data to path /home/ec2-user/uclresearchanalysis/data/givenchy/pickle/30_data.dat
Finished dumping data to path /home/ec2-user/uclresearchanalysis/data/givenchy/pickle/30_data.dat
extracted 50675 of rows


,label,Spr_generation0,Spr_generation-1,Spr_timeSinceSeed0,Spr_timeSinceSeed-1,Spr_totalNodeInfected,Spr_totalFollowers,Spr_totalFriends,Spr_followersCentrality0,Spr_followersCentrality-1,...,Stat_average_favouritesCount,Stat_average_friendsCount,Stat_average_listedCount,Stat_average_normalizedUserStatusesCount,Stat_average_normalizedUserFollowersCount,Stat_average_normalizedUserFavouritesCount,Stat_average_normalizedUserListedCount,Stat_average_normalizedUserFriendsCount,Stat_max_kOut,Stat_min_kOut
0,1,1,2,93.14,112.70,1796,8338643,1391046,8338643,8338643,...,20235.500000,555.000000,225.000000,16.086683,5.687387,7.392444,0.093771,0.217664,27082,515
1,0,0,2,0.00,10.67,116,3014932,154438,3014932,3014932,...,6297.666667,7651.000000,4763.666667,13.814488,245.229235,4.219706,1.420824,3.037090,2255118,8393
2,0,0,2,0.00,10.67,599,3755464,538640,3755464,3755464,...,6297.666667,7651.000000,4763.666667,13.814488,245.229235,4.219706,1.420824,3.037090,2255118,8393
3,0,0,0,0.00,0.00,956,5951130,807857,5951130,5951130,...,13137.800000,4783.800000,4939.400000,15.494473,253.821140,9.796335,1.501074,1.892361,2255118,8393
4,0,0,0,0.00,0.00,1414,7601061,1115589,7601061,7601061,...,13137.800000,4783.800000,4939.400000,15.494473,253.821140,9.796335,1.501074,1.892361,2255118,8393
5,0,0,0,0.00,0.00,1796,8338643,1391046,8338643,8338643,...,13137.800000,4783.800000,4939.400000,15.494473,253.821140,9.796335,1.501074,1.892361,2255118,8393
6,0,0,2,0.00,159.10,2144,12436777,1669491,12436777,12436777,...,12967.833333,4253.333333,4591.166667,17.622144,223.142762,8.801335,1.400879,1.661222,2255118,8393
7,0,0,2,0.00,159.10,2438,12885888,1857873,12885888,12885888,...,12967.833333,4253.333333,4591.166667,17.622144,223.142762,8.801335,1.400879,1.661222,2255118,8393
8,1,0,2,0.00,201.45,2727,13216334,2093647,13216334,13216334,...,11771.428571,3673.714286,3935.285714,15.313514,191.288305,8.101946,1.200754,1.447714,2255118,386
9,0,1,1,8.90,8.90,116,3014932,154438,3014932,3014932,...,11624.000000,21828.000000,999.000000,10.772162,79.288880,4.519440,0.388414,8.486781,225759,225759
